<a href="https://colab.research.google.com/github/joshuadollison/smallbizpulse/blob/jd-model/notebooks/model_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# SETUP: Mount Drive, Install Dependencies, Configure Styling
# ============================================================

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install VADER for sentiment analysis
!pip install vaderSentiment -q

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from collections import Counter
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# ── Consistent Plot Styling ──────────────────────────────────
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'figure.dpi': 120,
    'font.family': 'sans-serif',
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'axes.labelsize': 12,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'grid.linestyle': '--',
})

# SmallBizPulse color palette
COLORS = {
    'primary': '#2563EB',
    'secondary': '#F59E0B',
    'open': '#10B981',
    'closed': '#EF4444',
    'accent1': '#8B5CF6',
    'accent2': '#EC4899',
    'neutral': '#6B7280',
    'bg': '#F9FAFB',
}
PALETTE_OC = [COLORS['open'], COLORS['closed']]

print("Setup complete — libraries loaded, styling configured.")

In [ ]:
# ============================================================
# DATA LOADING
# ============================================================
# >>> UPDATE THIS PATH to match your Google Drive folder <<<
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/CIS509/yelp_dataset_new/'

def load_json(filename):
    filepath = DATA_PATH + filename
    with open(filepath, 'r') as f:
        first_char = f.read(1)
        f.seek(0)
        if first_char == '[':
            return pd.DataFrame(json.load(f))
        else:
            return pd.read_json(f, lines=True)

print("Loading datasets...")
business_df = load_json('yelp_academic_dataset_business.json')
print(f"  Business: {len(business_df):,} records")

review_df = load_json('yelp_academic_dataset_review.json')
print(f"  Review:   {len(review_df):,} records")

tip_df = load_json('yelp_academic_dataset_tip.json')
print(f"  Tip:      {len(tip_df):,} records")

checkin_df = load_json('yelp_academic_dataset_checkin.json')
print(f"  Checkin:  {len(checkin_df):,} records")

user_df = load_json('yelp_academic_dataset_user.json')
print(f"  User:     {len(user_df):,} records")

print("\nAll datasets loaded successfully.")